In [1]:

import os
from datetime import datetime, timezone

# ✅ Ensure outputs save relative to project root
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DOCS_DIR = os.path.join(BASE_DIR, "docs")
VIS_DIR = os.path.join(BASE_DIR, "visualizations", "risk_maps")

os.makedirs(DOCS_DIR, exist_ok=True)
os.makedirs(VIS_DIR, exist_ok=True)


In [2]:

import requests

today_str = datetime.now(timezone.utc).strftime("%Y%m%d")
tle_file = os.path.join(BASE_DIR, f"tle_data_{today_str}.txt")
url = 'https://celestrak.com/NORAD/elements/gp.php?GROUP=active&FORMAT=tle'

response = requests.get(url)
if response.status_code == 200:
    with open(tle_file, 'w', encoding='utf-8') as f:
        f.write(response.text)
    print(f"✅ TLE data saved to: {tle_file}")
else:
    raise Exception(f"Failed to download TLE. Status code: {response.status_code}")


✅ TLE data saved to: c:\Users\DELL\Desktop\Interplanix\Git\interplanix-ssa\global_satellite_risk_tool\tle_data_20250626.txt


In [3]:

from skyfield.api import load, wgs84
import pandas as pd

satellites = load.tle_file(tle_file)
ts = load.timescale()
now = ts.now()

records = []
for sat in satellites[:50]:
    try:
        geocentric = sat.at(now)
        subpoint = wgs84.subpoint(geocentric)
        lat = subpoint.latitude.degrees
        lon = subpoint.longitude.degrees
        alt = subpoint.elevation.km
        dist = abs(lat)

        risk = "Low" if dist <= 10 else "Medium" if dist <= 50 else "High"

        records.append({
            'name': sat.name,
            'latitude': lat,
            'longitude': lon,
            'altitude_km': alt,
            'dist_from_equator': dist,
            'risk_level': risk
        })
    except Exception as e:
        print(f"⚠️ Skipping {sat.name} due to error: {e}")

df = pd.DataFrame(records)
print(df.head())


           name   latitude   longitude  altitude_km  dist_from_equator  \
0   CALSPHERE 1  69.427366  -90.260854  1007.261662          69.427366   
1   CALSPHERE 2 -38.897187  -87.164887  1066.702490          38.897187   
2         LCS 1  14.624789  123.494352  2796.379265          14.624789   
3     TEMPSAT 1 -75.564363 -121.607183  1185.020260          75.564363   
4  CALSPHERE 4A  61.527157  151.349548  1088.554094          61.527157   

  risk_level  
0       High  
1     Medium  
2     Medium  
3       High  
4       High  


In [4]:

import plotly.graph_objects as go

color_map = {'Low': 'green', 'Medium': 'orange', 'High': 'red'}
df['color'] = df['risk_level'].map(color_map)

fig = go.Figure(go.Scattergeo(
    lon = df['longitude'],
    lat = df['latitude'],
    text = df['name'] + '<br>Risk: ' + df['risk_level'],
    mode = 'markers+text',
    marker = dict(size=6, color=df['color'], opacity=0.8, line=dict(width=0.5, color='black')),
    textposition='top center'
))

fig.update_geos(showland=True, landcolor="LightGray")
fig.update_layout(
    title='🛰️ Satellite Ground Track Risk Map',
    geo=dict(
        projection_type='orthographic',
        showcoastlines=True,
        showland=True,
        showocean=True,
        oceancolor='lightblue',
        landcolor='beige',
        showframe=False,
    )
)

# Save outputs
date_str = datetime.now().strftime("%Y-%m-%d")
fig.write_html(os.path.join(DOCS_DIR, "satellite_risk_map_sample.html"))
fig.write_html(os.path.join(VIS_DIR, f"{date_str}_satellite_risk_map.html"))
df.to_csv(os.path.join(VIS_DIR, f"{date_str}_risk_data.csv"), index=False)

print("✅ All files saved to project root directories.")


✅ All files saved to project root directories.
